<a href="https://colab.research.google.com/github/drbarak/NLP-FinalProject/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
INSTALL = False
if INSTALL:
  # to switch version of spacy
  !pip uninstall spacy
  !pip uninstall en_core_web_mdy
  
  !pip install spacy==2.2.4 #3.0.6

  !pip install pyforest
  #!pip install wget
  !python -m spacy download en_core_web_md

# INIT

In [3]:
from importlib import import_module
p = print
def my_import(lib, package=None, func=None, version=None):
  try:
    import_module(lib)
  except:
    if package is None: package = lib
    if version:
      p('pip install "{package}=={version}"')
      !pip install "{package}=={version}"
    else:
      p('pip install "{package}"')
      !pip install "{package}"
    import_module(lib)
  
  if func is not None:
    return import_module(lib).__getattribute__(func)

In [ ]:
my_import('pyforest')
import json, bs4
import string
import random
import time, pytz
from datetime import datetime
tz = pytz.timezone('Israel')

ic = my_import('icecream', func='ic')
ic.configureOutput(includeContext=True)  # include module name and line number

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
space_punct_dict = dict((ord(punct), ' ') for punct in string.punctuation)

p = print
d = display

In [ ]:
#my_import('spacy', version='3.0.6')
p('spacy.__version__', spacy.__version__)
from spacy.tokens import Doc, Span, Token
from spacy.matcher import PhraseMatcher
from spacy.language import Language

# Load md (medium) English tokenizer, tagger, parser and NER, and vectors(vectors are not included in the 'sm' model)
#nlp = spacy.load('en_core_web_md')  NOT WORKING ON COLAB
try:
  import en_core_web_md
except:  
  !python -m spacy download en_core_web_md
  import en_core_web_md
  
nlp = en_core_web_md.load()

In [6]:
nltk.download(['punkt', 'stopwords'])
stopwords = nltk.corpus.stopwords.words('english')

<IPython.core.display.Javascript object>

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


<IPython.core.display.Javascript object>

In [ ]:
#!pip uninstall googletrans
my_import('googletrans', version='3.1.0a0')
from googletrans import Translator
translator = Translator()

In [8]:
import requests

api_key = 'c2adfa29edfd95ad16efab9218619ff3'
URL = "http://api.openweathermap.org/data/2.5/{0}?"
icon_url = " http://openweathermap.org/img/wn/{0}@2x.png"  # 10d



In [9]:
path = '/content/drive/MyDrive/Project5_ChatBot/'
data_url = 'https://raw.githubusercontent.com/DanielKorenDataScience/NLP-FinalProject/main/data/' 

In [10]:
TEST = False
LANG = 'en'

In [11]:
def process_text(text):
    doc = nlp(text.lower())
    result = []
    for token in doc:
        if token.text in nlp.Defaults.stop_words:   # len(nlp.Defaults.stop_words)=326
            continue
        if token.is_punct:
            continue
        if token.lemma_ == '-PRON-':
            ic('-PRON-: ', token.lemma_)
            continue
        result.append(token.lemma_)
    return " ".join(result)

In [12]:
def calculate_similarity(text1, text2):
    def get_similarity(doc1, doc2):
      if doc1 and doc1.vector_norm:
        if doc2 and doc2.vector_norm:
          return doc1.similarity(doc2)
      return 0

    doc1 = nlp(text1)
    #ic(doc1.pos_)
    doc2 = nlp(text2)
    similarity1 = get_similarity(doc1, doc2)

    doc1 = nlp(process_text(text1))
    #ic(doc1)
    doc2 = nlp(process_text(text2))
    #ic(doc2)
    similarity2 = get_similarity(doc1, doc2)

    return max(similarity1, similarity2)

# Get cities and update model

In [13]:
GIT = True
git_path = 'https://raw.githubusercontent.com/DanielKorenDataScience/NLP-FinalProject/main/data/'

def get_json(fname, GIT=True):
  p(fname)
  if GIT:
    resp = requests.get(data_url + fname)
    return json.loads(resp.text)
  with open(path + fname, encoding="utf8") as f:
    return json.loads(f.read())

In [14]:
def load_cities_il():
  path_ = path
  fname = 'cities_IL.csv'
  if False and GIT:
    path_ = git_path
  df = pd.read_csv(path_ + fname, names=['city', 'district']) # the 'district' are recognized by the api
  df.city = df.city.str.translate(remove_punct_dict)
  CITIES = df.city.tolist()
  df = df.set_index('city')
  return df, CITIES
cities_il_df, CITIES_IL = load_cities_il()
cities_il_df.tail()

<IPython.core.display.Javascript object>

,district
city,
galil,northern district
sharon,netanya
north,northern district
central,netanya
center,netanya


In [15]:
UPD_IL = False
if UPD_IL:
  #CITIES_API = get_json('city.list.json')
  CITIES_API = CITIES_API[:10]

  df = pd.DataFrame(CITIES_API)
  df = df.drop(['id'], axis='columns')
  df.name = df.name.str.lower()
  df.name = df.name.str.translate(remove_punct_dict)

  CITIES_API_city = sorted(set(df.name.to_list()))
  p(CITIES_API_city)

  #p(CITIES_API.values())
  #for c in CITIES_API
  'Ḩeşār-e Sefīd'.lower().translate(remove_punct_dict) in CITIES_API_city#CITIES_API[0].values()

In [16]:
#my_import('wget')
#import wget

def load_CITIES_API():
  # file too big for git so read it from my google drive using a shared link
  CITIES_API = get_json('city.list.json')
  ''' 
  git_path = 'https://raw.githubusercontent.com/DanielKorenDataScience/NLP-FinalProject/main/' + 'city.list.json'
  resp = requests.get(git_path)
  CITIES_API = json.loads(resp.text)

  google_shared_link = 'https://drive.google.com/file/d/19lB4T32o1VbFAKRk_PExbNYV8mcZ8hRT/view?usp=sharing'
  url = 'https://drive.google.com/uc?authuser=0&id=19lB4T32o1VbFAKRk_PExbNYV8mcZ8hRT&export=download'
  file_id = '19lB4T32o1VbFAKRk_PExbNYV8mcZ8hRT'
  
  while True:
    try:
      with open('city.list.json', encoding="utf8") as f:
        CITIES_API = json.loads(f.read())
      break
    except:
      wget.download(url)  # download the file to local google drive with the original name in the share link
      # now can open with no path because it is in the local 'colab' directory
  
  '''
  # add all israeli cities so when looking for 'north' gets multiple cities because there is one in France
  # and not to get an error is specify 'north, israel' becuse the country does not match
  df = pd.DataFrame(CITIES_API)
  df.name = df.name.str.lower()
  df.name = df.name.str.translate(remove_punct_dict)
  CITIES_API_city = sorted(set(df.name.to_list()))

  for city in CITIES_IL:
    CITIES_API.append({'name': city, 'country': 'IL', 'state': '', 'coord': ''})
  
  df = pd.DataFrame(CITIES_API)
  df.name = df.name.str.lower()
  df.name = df.name.str.translate(remove_punct_dict)
  df = df.drop(['id'], axis='columns')

  CITIES_API_city = sorted(set(df.name.to_list()))
  CITIES_API_country = sorted(set(df.country.str.lower().to_list()))
  if CITIES_API_country[0] == '':
    CITIES_API_country = CITIES_API_country[1:]
  CITIES_API_state = sorted(set(df.state.str.lower().to_list()))
  if CITIES_API_state[0] == '':
    CITIES_API_state = CITIES_API_state[1:]
  if CITIES_API_state[0] == '00':
    CITIES_API_state = CITIES_API_state[1:]
  
  df.set_index('name', inplace=True)
  #d(df.head)
  return df, CITIES_API_city, CITIES_API_country, CITIES_API_state

df_CITIES_API, CITIES_API_city, CITIES_API_country_code, CITIES_API_state_code = load_CITIES_API()
df_CITIES_API.head()
#df_CITIES_API.loc['taglag'].coord

city.list.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,state,country,coord
name,,,
ḩeşāre sefīd,,IR,"{'lon': 47.159401, 'lat': 34.330502}"
‘ayn ḩalāqīm,,SY,"{'lon': 36.321911, 'lat': 34.940079}"
taglag,,IR,"{'lon': 44.98333, 'lat': 38.450001}"
qabāghlū,,IR,"{'lon': 46.168499, 'lat': 36.173302}"
‘arīqah,,SY,"{'lon': 36.48336, 'lat': 32.889809}"


In [17]:
def load_countries():
  path_ = path  
  fname = 'countries.csv'
  if GIT:
    path_ = git_path
  df = pd.read_csv(path_ + fname)
  df.Name = df.Name.str.lower()
  COUNTRIES = df.Name.tolist()
  df = df.set_index('Code')
  return df, COUNTRIES

countries_df, COUNTRIES = load_countries()
countries_df.head()

<IPython.core.display.Javascript object>

,Name
Code,
AF,afghanistan
AX,åland islands
AL,albania
DZ,algeria
AS,american samoa


In [18]:
def load_capitals():
  path_ = path
  fname = 'capitals.json'
  if GIT:
    path_ = git_path
  CAPITALS = pd.read_json(path_ + fname, encoding="utf8", typ='series').str.lower()

  CAPITALS.index = CAPITALS.index.str.lower()
  df = pd.DataFrame([CAPITALS])
  df = df.T
  df = df.reset_index()
  df.columns = ['country', 'capital']
  df = df.set_index('capital')
  return df, CAPITALS.to_dict()
capitals_df, CAPITALS = load_capitals()

p(CAPITALS['albania'])
p(capitals_df.loc['madrid'].country)
capitals_df.head()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

tirana
spain


,country
capital,
kabul,afghanistan
mariehamn,åland islands
tirana,albania
algiers,algeria
pago pago,american samoa


In [19]:
def load_largest():
  fname = '100-largest-cities.csv'
  path_ = path
  if GIT:
    path_ = git_path
  df = pd.read_csv(path_ + fname, encoding='ISO-8859-8')
  df.city = df.city.str.lower()
  df.city = df.city.str.translate(remove_punct_dict)
  LARGEST = df.city.tolist()
  df.country = df.country.str.lower()
  df = df.set_index('city')
  return df, LARGEST
largest_df, LARGEST = load_largest()

d(largest_df.head())
p(largest_df.loc['toronto'].country)

<IPython.core.display.Javascript object>

,country
city,
tokyo,japan
delhi,india
shanghai,china
so paulo,brazil
ciudad de mxico mexico city,mexico


canada


In [20]:
def load_actions():
  path_ = path
  fname = 'actions.json'
  actions = get_json(fname, False)
  ACTIONS_patterns_ = [y for tag in actions["actions"] for y in tag['patterns']]
  ACTIONS_ = {y: tag['tag'] for tag in actions["actions"] for y in tag['patterns']}
  ACTIONS_tags_ = {tag['tag']:'' for tag in actions["actions"]}
  return ACTIONS_patterns_, ACTIONS_, ACTIONS_tags_
ACTIONS_patterns, ACTIONS, ACTIONS_tags = load_actions()

p(ACTIONS_patterns[-5:])
p(list(ACTIONS.items())[:5])
p(ACTIONS.keys())
p(ACTIONS_tags)
ACTIONS['hot']
ACTIONS_tags.keys()
'clouds' in ACTIONS

actions.json
['evening', 'sundown', 'cloud', 'clouds', 'fahrenheit']
[('weather', 'all'), ('tornedo', 'weatherWind'), ('hurricane', 'weatherWind'), ('gale', 'weatherWind'), ('gust', 'weatherWind')]
dict_keys(['weather', 'tornedo', 'hurricane', 'gale', 'gust', 'whirlwind', 'cyclone', 'fog', 'mist', 'isobar', 'barometric', 'heat wave', 'freeze', 'climate', 'conditions', 'forecast', 'outlook', 'thunder', 'thunderstorm', 'storm', 'rain', 'drizzle', 'snow', 'dry', 'rainbow', 'overcast', 'shower', 'lightning', 'blustery', 'downpour', 'tropical', 'water cycle', 'drought', 'hail', 'icicle', 'flood', 'muggy', 'flash flood', 'atmospheric', 'cold front', 'cold snap', 'condensation', 'ice storm', 'snowfall', 'air', 'balmy', 'avalanche', 'frost', 'blizzard', 'spring', 'smog', 'ozone', 'autumn', 'winter', 'summer', 'sleet', 'sky', 'easterlies', 'westerlies', 'dew', 'surge', 'monsoon', 'permafrost', 'temperature', 'hot', 'warn', 'cold', 'temp', 'degree', 'thermal', 'heat', 'heat index', 'warm', 'pres

True

In [21]:
def load_test():
  path_ = path
  fname = 'test_questions.txt'
  questions = get_json(fname, False)
  return questions
QUESTIONS = load_test()

p(QUESTIONS)
QUESTIONS[0]

test_questions.txt
['Weather forecast in Tel aviv', "'קשאיקר ןמ ןדרשקך", 'How hot is it in Herzliya', 'Please send the log to drbarak@talkie.co.il', 'Today’s temperature in Netanya', 'Moist in Hadera', 'How many degrees tonight in Hadera Ofakim', 'Weekly forecast for Ramat Hasharon', 'Israel desert forecast', 'Weather in Ramat gan', 'Moist in Givatayim', 'Wave height in Ashdod', 'Moist ashkelon', 'Night degrees in Eilat', 'Newt 10 days forecast in Herzliya and Raanana', 'Is there a weather forecast on Herzliya', 'Would like to get weather forecast on Hadera next week', 'North Israel weather', 'Monthly average forecast for north israel', 'Monthly average forecast on Thailand', 'What are the optional cities for weather forecast', 'Who is the first prime minister of Israel', 'Who where the 3 stooges', 'What is a banana', 'Who was Napoleon', 'When was photography invented', 'Are there aliens']


'Weather forecast in Tel aviv'

In [232]:
dayInWeek={'sunday':1,'monday':2,'tuesday':3,'wednesday':4,'thursday':5,'friday':6,'saturday':7}
dateToNum={"today":0,"tomorrow":1,'twoDays':2 }
textToNumbers={'one':1,'two':2,'three':3,'four':4,'five':5,'six':6,'seven':7}

In [107]:
def load_dates():
  fname = 'datesWords.txt'
  path_ = path
  if False and GIT:
    path_ = data_url
    print(path_)
  df = pd.read_csv(path_ + fname)
  #a_series = pd. Series(['','','','in 1 week'], index = df. columns)
  #df = df.append(a_series, ignore_index=True)
  df.fillna('', inplace=True)
  return df
datesDf= load_dates()
listNlp = {}
for column in datesDf:
  listNlp.update({column: [nlp.make_doc(t) for t in datesDf[column].tolist() if t != '']})
#p(listNlp)
datesDf

<IPython.core.display.Javascript object>

,today,tomorrow,twoDays,thisweek,nextWeek
0,now,tomorrow,the day after tommorow,this week,next week
1,today,next day,,the weekend,week later
2,tonight,following day,,,following week
3,this morning,one day,,this weekend,next weekend
4,current,,,,week later
5,right now,,,,coming week
6,,,,,next weekend


## Messages

In [24]:
def load_messages():
  path_ = path
  fname = 'messages.json'
  messages = get_json(fname, False)
  intents = {lang: messages["intents"]["language"][lang] for lang in messages["intents"]["language"]}
  #MESSAGES_lang = [lang for lang in messages["messages"]["language"]]
  MESSAGES = {lang: messages["messages"]["language"][lang] for lang in messages["messages"]["language"]}
  MESSAGES_lang = list(MESSAGES.keys())
  MESSAGES_en, intents_en = [], []
  if 'en' in MESSAGES:
    MESSAGES_en = MESSAGES['en']["messages"]
    for part in intents['en']["messages"]:
      intents_en.append(part["responses"])
  # read also the new_mssages
  
  try:
    with open(path + 'new_messages.json', "r") as f:
      list_d = json.loads(f.read())
  except Exception as error:
    list_d = []
  if len(list_d) > 0:
    for dic in list_d:
      intents.update(dic['intents']['language'])
      MESSAGES.update(dic['messages']['language'])
    MESSAGES_lang = list(MESSAGES.keys())
  return MESSAGES_lang, MESSAGES, MESSAGES_en, intents, intents_en
MESSAGES_lang, MESSAGES, MESSAGES_en, intents, intents_en = load_messages()
p(MESSAGES_lang)
p(intents_en)
#intents['he']["messages"][0]
intents_en[1]

messages.json
['en', 'he', 'fr', 'es', 'ar', 'ru']
[["Hi. My name is WeatherBot and I'm a chatbot. If you want to exit, type 'Bye' If you want help type 'Help'"], ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], ['Happy to help!', 'Any time!', 'My pleasure'], ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], ['I can give you information about the weather in cities around the world, in your language. You can ask about clouds, rain, temp, pressure, humidity, wind, visibility, sunrise and upto 7 days from today'], ['WeatherBot', 'Ask Bot', 'Debug', 'User Input', 'You', 'Bot', 'ID', 'Conversation'], ['Done', 'sent']]


['See you!', 'Have a nice day', 'Bye! Come back again soon.']

In [25]:
def set_capitals():
  # Getter that looks up the span text in the dictionary of country capitals
  get_capital = lambda span: CAPITALS.get(span.text)

  # Register the Span extension attribute "capital" with the getter get_capital
  Span.set_extension("capital", getter=get_capital, force=True)

  # Process the text and print the entity text, label and capital attributes
  doc = nlp("Czech Republic may help Slovakia protect its airspace")
  print([(ent.text, ent.label_, ent._.capital) for ent in doc.ents])

# Call the web site

{"coord":{"lon":34.8,"lat":32.0833},

"weather":[{"id":801,"main":"Clouds","description":"few clouds","icon":"02n"}],

"base":"stations",

"main":{"temp":295.69,"feels_like":295.83,"temp_min":294.42,"temp_max":296.58,"pressure":1016,"humidity":70},

"visibility":10000,

"wind":{"speed":2.06,"deg":10},

"clouds":{"all":20},

"dt":1623961727,

"sys":{"type":1,"id":6845,"country":"IL","sunrise":1623897263,"sunset":1623948546},

"timezone":10800,

"id":293396,

"name":"Tel Aviv",

"cod":200}

In [223]:
def call_web(city_name, country_code, fahrenheit, one_call):
    state = ''
    api_url = f"{URL}q={city_name},{state},{country_code}&units={'metric' if not fahrenheit else 'imperial'}&lang={LANG}&appid={api_key}".format('weather')
    #ic(city_name, country_code, fahrenheit, count)
    if one_call:
      cor = df_CITIES_API.loc[city_name].coord
      if type(cor) is not dict: #more than one record for that city
        for index, row in df_CITIES_API.loc[city_name].iterrows():
          #ic('Z:', row['country'], row['coord'], country_code==row['country'])
          if row['coord'] != '' and (country_code==row['country'] or country_code == ''):
            cor = row['coord']
            break
      api_url = f"{URL}lat={cor['lat']}&lon={cor['lon']}&exclude=minutely,hourly&units={'metric' if not fahrenheit else 'imperial'}&lang={LANG}&appid={api_key}".format('onecall')

    ic(api_url)

    response = requests.get(api_url)
    response_dict = response.json()

    weather = response_dict #["weather"]#[0]#["description"]

    if response.status_code == 200:
        return weather
    elif response.status_code == 404:
        return None
    else:
        print(MESSAGES[LANG]["messages"]["web_error"].format(response.status_code, api_url))
        ic(response_dict['message'])
        return None

In [222]:
def get_weather_info(actions, one_call, cnt, city_weather, default, LANG, DISP):
  all_action = actions[0][0] == 'all'  # always first
  #one_call, cnt, city_weather)
  result = ''
  for action in actions:
    tag = action[0]
    if tag == "fahrenheit": continue
    user_act = action[1]
    if not one_call:
      data = city_weather if tag == "all" else city_weather['main'] \
        if tag in ["weatherTemp", "weatherPressure", 'temp', "pressure", "humidity"] else \
              city_weather['sys'] if tag == "sunrise" else city_weather['wind'] \
              if tag in ["weatherWind", "wind"] else city_weather[tag] 
    else:
      data = city_weather
    if DISP: ic('in loop', user_act, tag, data)
    if tag in ['clouds']:
      clouds_percentage = data['all'] if not one_call else data['clouds']
      if clouds_percentage < 10: # no clouds
        if 'clear sky' not in default.lower():
          result += f'{MESSAGES[LANG]["messages"]["clear sky"]}, '
        elif not all_action:
          result += f' {default},'
      elif default not in result:
        result += f' {default},'
      if all_action: continue
    if user_act == "rain": 
      if user_act in default.lower():
        if default not in result:
          result += f' {default},'
      else:
        result += f' {MESSAGES[LANG]["messages"]["no"]} {user_act},'
      if all_action: continue
    if all_action and tag != 'all': continue
    if tag in ["weather", "weatherPressure","weatherWind","weatherTemp","all"]:
      if default not in result:
        result += f' {default},'
    if tag in ["weatherTemp", "weatherPressure", 'temp', "pressure", "humidity", "all"]:
      data_tag = data['main'] if tag == 'all' and not one_call else data
      #one_call, tag, data_tag, data)
      if tag in [ "weatherTemp", "temp", "all"]:
        for i, temp in enumerate(MESSAGES[LANG]["messages"]["temp"]):
          if i == 0: 
            if cnt == 0:
              info = data_tag["temp"]
            else:
              info = data_tag["temp"]['day']
          elif i == 1: 
            if cnt == 0:
              info = data_tag["feels_like"]
            else:
              info = data_tag["feels_like"]['day']
            if one_call  and cnt == 0:
              break
          elif i == 2: info = data_tag["temp_min"] if cnt == 0 else data_tag["temp"]["min"]
          elif i == 3: info = data_tag["temp_max"] if cnt == 0 else data_tag["temp"]["max"]
          result += f' {temp} = {info},'
      if tag in ["weatherPressure", "pressure", "all"]:
        result += f' {MESSAGES[LANG]["messages"]["pressure"]} = {data_tag["pressure"]},'
      if tag in ["humidity", "all"]:
        result += f' {MESSAGES[LANG]["messages"]["humidity"]} = {data_tag["humidity"]},'
    if tag in ["wind", "weatherWind", "all"]:
      data_tag = data['wind'] if tag == 'all' and not one_call else data
      for i, wind in enumerate(MESSAGES[LANG]["messages"]["wind"]):
        if i == 0: info = data_tag["speed"] if not one_call else data_tag["wind_speed"]
        elif i == 1: info = data_tag["deg"] if not one_call else data_tag["wind_deg"]
        elif i == 2: 
          if "gust" in data_tag: 
            info = data_tag["gust"] 
        elif "wind_gust" in data_tag: 
            info = data_tag["wind_gust"] 
        else: 
            continue
        result += f' {wind} = {info},'
    if tag in ["visibility","all"]:
      if cnt > 0 and 'visibility' not in data:
        data_tag = MESSAGES[LANG]["messages"]["no_info"]
      else:
        data_tag = data['visibility'] if tag == 'all' or one_call else data
      result += f' {MESSAGES[LANG]["messages"]["visibility"]} = {data_tag},'
    if tag in ["sunrise","all"]:
      data_tag = data['sys'] if tag == 'all' and not one_call else data
      for i, wind in enumerate(MESSAGES[LANG]["messages"]["sunrise"]):
        if i == 0: info = data_tag["sunrise"]
        elif i == 1: info = data_tag["sunset"]
        result += f' {wind} = {time.strftime("%H:%M:%S", time.gmtime(info + 3 * 60 * 60))},'
  return result

In [235]:
def get_weather(days, location, actions, range_days, LANG=LANG, DISP=False):
  #dt, location)
  city_w = city = location[0][0]  # assume one city (not handling if there are 2 cities)
  country = location[0][1]
  #city, city_w)
  result = ''

  #convert israeli cities to district that the API knows ('sharon' is in API because there is such city in Australia)
  if country == 'israel' and city in cities_il_df.index.values:
    city_w = cities_il_df.loc[city].district

  #city_w, location)
  #country, len(countries_df.Name))
  if country in list(countries_df.Name):
    country_code = countries_df[countries_df.Name == country].index[0]
  else:
    country_code = ''
  #f'Y: {city_w}, [{country_code}]')

  fahrenheit = True if len([action for action in actions if action[0] == "fahrenheit"]) > 0 else False
  #p('in get_weather:', fahrenheit)
  #p(city_w, country_code, fahrenheit, one_call)
  one_call = (days > 0 or range_days > 1)
  p(one_call)
  city_weather_resp = call_web(city_w, country_code, fahrenheit, one_call)

  if DISP: ic(city_weather_resp)
  if DISP: ic('get_w', actions)
  if city_weather_resp is None:
    return MESSAGES[LANG]["messages"]["api_error"], ''
    # check what informtion the user asked for

  city_weather = city_weather_resp if not one_call else city_weather_resp["current"]

  default = city_weather["weather"][0]["description"]
  icon = city_weather["weather"][0]["icon"]
  url_icon = f"{icon_url}".format(icon)
  if DISP: ic(url_icon)
  if DISP: ic('get_w', default)
  result = ''

  for cnt in range(days, 9):
    if cnt > 0: 
      # bug in python - cannot have r += z + x if cond else y
      M = MESSAGES[LANG]["messages"]["web_days_msg"].format(cnt) if cnt > 1 else MESSAGES[LANG]["messages"]["web_day_msg"]
      result += '<br>' + M
      city_weather = city_weather_resp["daily"][0]
    new_result = get_weather_info(actions, one_call, cnt, city_weather, default, LANG, DISP)
    #cnt, new_result)
    result += new_result
    if not one_call: break

  if result == '': result = default
  if result[-1] == ',': result = result[:-1]
  country = location[0][1]
  if LANG != 'en': # in case there are parts of the answer which are not in the proper langauge
    response = translator.translate(city, dest=LANG) # need to separet ity from country in case one of them has multi words and google doe not keep the qoutation mark as needed
    #response = translator.translate(f"({city}, {country})", dest=LANG)
    #city, country = response.text
    p(f"{response.origin} ({response.src}) --> {response.text} ({response.dest})")
    city = response.text
    response = translator.translate(country, dest=LANG)
    country = response.text
  prompt = MESSAGES[LANG]["messages"]["web_msg"] if days == 0 else MESSAGES[LANG]["messages"]["web_msg_no_current"]
  response =  prompt.format(city.capitalize(), country.capitalize(), result)
  return response, url_icon
r = '''    
#answer = get_weather('current', [('jerusalem', 'israel')], [('clouds', 'cloud'), ('temp', 'temp'), ('pressure', 'pressure'), ('humidity', 'humidity'), ('wind', 'wind'), ('visibility', 'wind'), ('sunrise', 'wind'), ('weather','rain')])
answer = get_weather('current', [('jerusalem', 'israel')], 
  [('temp', 'heat'),
  ('weather', 'rain'),
  ('temp', 'temp'),
  ('pressure', 'pressure'),
  ('humidity', 'humidity'),
  ('wind', 'wind'),
  ('visibility', 'visibility'),
  ('sunrise', 'sunrise')])
'''
#{'date': 'current', 'action': ('weather', 'weather'), 'city': [('vi', 'sweden')]}
# if tag in ["weatherTemp", "weatherPressure", 'temp', "pressure", "humidity", "all"]:

#answer = get_weather('current', [('toronto', 'canada')],  [('all', 'all'), ('visibility', 'visibility'), ('sunrise', 'sunrise'), ('temp', 'heat'), ('wind', 'wind'), ('weather','rain'), ('fahrenheit','fahrenheit')], DISP=False)
#answer

# NLM

In [29]:
FAST_START = False

In [206]:
get_parts("weather currently in tel aviv", True)


Tokens of [weather]: dep_=ROOT, ent_type=, head=weather, lemma_=weather, pos_=NOUN, tag_=NN, is_action=True
Tokens of [currently]: dep_=advmod, ent_type=DATE, head=in, lemma_=currently, pos_=ADV, tag_=RB, is_action=False
Tokens of [in]: dep_=prep, ent_type=, head=weather, lemma_=in, pos_=ADP, tag_=IN, is_action=False
Tokens of [tel]: dep_=compound, ent_type=GPE, head=aviv, lemma_=tel, pos_=PROPN, tag_=NNP, is_action=False
Tokens of [aviv]: dep_=pobj, ent_type=GPE, head=in, lemma_=aviv, pos_=PROPN, tag_=NNP, is_action=False


<IPython.core.display.Javascript object>

Entities of [currently]: DATE='Absolute or relative dates or periods', is_city=True, is_country=False


<IPython.core.display.Javascript object>

Entities of [tel aviv]: GPE='Countries, cities, states', is_city=False, is_country=False
{'date': ['currently'], 'time': [], 'action': [('all', 'weather')], 'state': ['tel aviv']}


{'action': [('all', 'weather')],
 'date': ['currently'],
 'state': ['tel aviv'],
 'time': []}

In [213]:
nlp = en_core_web_md.load()
'''
 becuase it takes long time to init (about 1.5 minutes), and once a day the PythonAnywhere bring down all sites,
 for maintenance, and to prevet the first user from waiting too long, I adde a line in the nightly batch file
 rungui.bat that start the site atabout 3 AM
  start chrome http://drbarak.pythonanywhere.com/chatbot
'''
def update_nlp():
  matcher = PhraseMatcher(nlp.vocab)
  date_time = ['now', 'current', 'currently']
  date_time_str = "now, current, currently"
  CITIES = set(CITIES_IL + CITIES_API_city + date_time)# + ACTIONS_patterns) it recognized all the actions as GPE
  #CITIES = list(CITIES)[:10]
  #CITIES)
  # disable other pipes while doing the traing to speed it up - went down from 1:40 to 0:40 minutes
  if not FAST_START:
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):
      matcher.add("CITIES", None, *list(nlp.pipe(CITIES)))
#      matcher.add("ACTIONS", None, *list(nlp.pipe(ACTIONS_patterns)))

  # country codes causes problems because many are similar to regular words (eg. 'IN' similar to 'in')
  #matcher.add("CITIES_API_country_code", None, *list(nlp.pipe(CITIES_API_country_code)))
  # due to duplicates with country codes ('IL' = Israel, and Ilinoie), we have to check manually for states
  #matcher.add("CITIES_API_state_code", None, *list(nlp.pipe(CITIES_API_state_code)))

  # Register the Span extension attributes "is_country" and "is_city" 
  check_country = lambda span: span.text.lower().translate(remove_punct_dict) in COUNTRIES
  Span.set_extension('is_country', getter=check_country, force=True)

  check_city = lambda span: span.text.lower().translate(remove_punct_dict) in CITIES
  Span.set_extension('is_city', getter=check_city, force=True)

  check_action = lambda token: token.text.lower().translate(remove_punct_dict) in ACTIONS_patterns or \
                                token.lemma_.lower().translate(remove_punct_dict) in ACTIONS_patterns
  Token.set_extension('is_action', getter=check_action, force=True)

  pipes = nlp.pipe_names
  if 'gpe_component' in pipes:
    nlp.remove_pipe('gpe_component')

  if spacy.__version__ > '3':
    @Language.component("gpe_component")
    def gpe_component(doc):
        # Create an entity Span with the label "GPE" for all matches
        matches = matcher(doc)
        try:
          doc.ents = [Span(doc, start, end, label="GPE") for match_id, start, end in matches]
          #if len(doc.ents) > 0:
          #  doc.ents, 'rain' in CITIES)
          #doc.ents += tuple([Span(doc, start, end, label="WACT") for match_id, start, end in matches if nlp.vocab.strings[match_id] == 'ACTIONS'])
        except ValueError as e: #[E103] Trying to set conflicting doc.ents: '(2, 4, 'GPE')' and '(3, 4, 'GPE')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap.
          if 'E103' in e.args[0]:
            return doc  # use the standard model of Spacy, hopefully it works (as in the case "weather in new york") or  other rounds will get the proper result
          raise
        return doc

    # Add the new component to the pipeline
    nlp.add_pipe('gpe_component', first=True)
  else:
    def gpe_component(doc):
        # Create an entity Span with the label "GPE" for all matches
        matches = matcher(doc)
        try:
          #doc.ents = [Span(doc, start, end, label="GPE") for match_id, start, end in matches if nlp.vocab.strings[match_id] == 'CITIES']
          doc.ents = [Span(doc, start, end, label="GPE") for match_id, start, end in matches if doc[start:end].text not in date_time_str]
          doc.ents += tuple([Span(doc, start, end, label="DATE") for match_id, start, end in matches if doc[start:end].text in date_time_str])
          '''
          doc_ents = []
          for match_id, start, end in matches:
            ent = doc[start:end].text
            p(ent, start, end, ent in date_time_str)
            if ent in date_time_str:
              ent = Span(doc, start, end, label="DATE")
            else:
              ent = Span(doc, start, end, label="GPE")
            doc_ents.append(ent)
          doc.ents = doc_ents
          '''
#          if len(doc.ents) > 0:
#            doc.ents, 'rain' in CITIES)

#          doc.ents += tuple([Span(doc, start, end, label="WACT") for match_id, start, end in matches if nlp.vocab.strings[match_id] == 'ACTIONS'])
        except ValueError as e: #[E103] Trying to set conflicting doc.ents: '(2, 4, 'GPE')' and '(3, 4, 'GPE')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap.
          if 'E103' in e.args[0]:
            return doc  # use the standard model of Spacy, hopefully it works (as in the case "weather in new york") or  other rounds will get the proper result
          raise
        return doc

    # Add the new component to the pipeline
    nlp.add_pipe(gpe_component, first=True)
  #print(nlp.pipe_names)

update_nlp()
#get_parts("humidity nenita ellerbach", True)

<IPython.core.display.Javascript object>

# Get Parts (POS)

## check date

In [31]:
def remove_stop_words(doc1_list):
  user_list = []
  for doc1 in doc1_list:
    user_msg, space  = '', ''
    for token in doc1.split():
      if token in stopwords: continue
      text = token.lower().translate(remove_punct_dict)
      if text == '': continue
      user_msg += space + text
      space = ' '
    user_list.append(user_msg)
  return user_list
p(remove_stop_words(['friday']))
p(remove_stop_words(['friday','in 2 weeks']))

['friday']
['friday', '2 weeks']


In [32]:
TODAY = 0 # so can test different days

In [122]:
NO_DAYS = -999
def get_delta_days_weekday(text, dayInText_org=-NO_DAYS, weekend = False, range_days = 0, disp=False):
  if disp: p('get_delta_days_weekday', dayInText_org)
  days, week, dayInText = NO_DAYS, -1, dayInText_org
  if TODAY == 0:
    todayDate = datetime.now(tz)
    #todayDayWeek=(todayDate.weekday()+2-7) if (todayDate.weekday()+2>7) else  (todayDate.weekday()+2) # get day of week today
    #todayDayWeek = (todayDate.weekday() + 1) % 7 + 1 # get day of the week such that sunday=1,.. saturday=7
    todayDayWeek = todayDate.weekday() + 2 if todayDate.weekday() > 1 else 7 # from day of the week such that sunday=1,.. saturday=7
  else: 
    todayDayWeek = TODAY
  if disp: p(todayDayWeek)

  if text in list(dayInWeek.keys()): # if there is only one word of day in week
    if dayInText_org == -NO_DAYS: # if we did not find a day already
      dayInText = dayInWeek[text]
      if disp: p(text, dayInText, dayInWeek)
      if todayDayWeek <= dayInText:
        days = dayInText - todayDayWeek
      else:
        days = dayInText - todayDayWeek + 7
  else:# if there are additional word like 'next sunday'
    doc = nlp(text)
    for entity in doc:
      if disp: print(f'entity = {entity.text}, lemma_ = {entity.lemma_}')
      if entity.text in list(dayInWeek.keys()):
        dayInText = dayInWeek[entity.text.lower()]
        if todayDayWeek <= dayInText:
          days = dayInText - todayDayWeek
        else:
          days = dayInText - todayDayWeek + 7
          week = 0
      elif entity.lemma_ == 'next' and week == -1:
        if disp: print('get next')
        week = 1
      elif entity.lemma_ == 'weekend' and not weekend and week == -1:
        week = 1
        range_days = 2
        weekend = True
  if week == -1:
    week = 0
  if disp: p('get_delta_days_weekday: dayInText =', dayInText)
  return days, week, todayDayWeek, dayInText, weekend, range_days          


In [34]:
p(get_delta_days_weekday("friday"))
get_delta_days_weekday("sunday")

(6, 0, 7, 6)


(1, 0, 7, 1)

In [35]:
def get_number_period(doc, disp=False):
  if disp: p('get_number_period')
  days, week = NO_DAYS, 0
  for entity in doc:
    if disp: print(entity.pos_, entity.head, entity.lemma_)
    if entity.pos_=='NUM':
      if disp: print("found number "+entity.text)
      if entity.text.isdigit():
          num = int(entity.text)
      else:
          num = textToNumbers.get(entity.text, NO_DAYS)
      if disp: print ('number is '+str(num))
      if nlp(str(entity.head))[0].lemma_ == 'week': # if there is week add it to number to week and if not match it num of days
        if num != NO_DAYS:
          week = num
      else:
        days = num
  #if week > 0 and days == NO_DAYS:  days = 0
  return days, week        

In [230]:
def check_date(dateText, disp=False):
  days, week, found_days, cur_days = NO_DAYS, 0, NO_DAYS, NO_DAYS
  todayDayWeek = 0 # get day in week today
  dayInText = -NO_DAYS # get day in week by text
  f_string = " days={0}, week={1}, found_days={2}, cur_days={3}"
  range_days, weekend = 1, False # single day

  if len(dateText) == 0:
    return NO_DAYS, 0
  for text in dateText: # check if we get several date variables
    if disp: p('text =', text)
    #matcher = PhraseMatcher(nlp.vocab)
    for column in datesDf: # run on each column if there is match
      #listNlp = [nlp.make_doc(t) for t in df[column].tolist() if str(t) != 'nan']
      date_matcher = PhraseMatcher(nlp.vocab)
      date_matcher.add(column, None, *listNlp[column])
      #p(listNlp[column])
      doc = nlp(text.lower())
      matches = date_matcher(doc)
      if disp: p(column, matches)
      if len(matches) > 0: # if we find match in the list of column
        match_words = ''
        for match_id, start, end in matches:
          match_words += f"{doc[start:end].text}, "
        if disp:
          p('col:',column)
          print("matchFound", match_words)
        if column == "nextWeek": # check if there 
          if disp: print("is next Week")
          week = 1
          if days == NO_DAYS and found_days == NO_DAYS: days = 0
          if 'weekend' in match_words:
            range_days = 2
            weekend = True
        elif 'weekend' in match_words:
          range_days = 2
          weekend = True
        if len(dateText) == 1:        
          numFinal = dateToNum.get(column, NO_DAYS)
          if numFinal != NO_DAYS or days == NO_DAYS:
            return numFinal, range_days
        if disp: p(days, todayDayWeek, dayInText)
        # todayDayWeek is in the range 1=sunday to 7=saturday
        # dayInText is in the same range
        # in order to compare we need to convert both such the sunday = 8 (mon=1, tue=2...sat=7, sun=8)
        todayDayWeek_new = todayDayWeek if todayDayWeek > 1 else 8
        dayInText_new = dayInText if dayInText > 1 else 8
        if disp: p(days, todayDayWeek, dayInText, dayInText_new)
        if days != NO_DAYS:  # verify days match numOfdays (in case on Friday asked for "friday in 2 days" which is wrong)
          if column == "today" and days == 0:
            pass
          elif column == "tomorrow" and days == 1:
            pass
          elif column == "twoDays" and days == 2:
            pass
          elif column == 'thisweek' and (dayInText_new >= todayDayWeek_new):  # this week must be day after today up to sunday
            pass
          else:
            days = NO_DAYS
        cur_days = days  # days for this text independent of prev columns
        break # found a natch no need to check others
    if disp: print("1." + f_string.format(days, week, found_days, cur_days))
    if found_days == NO_DAYS: found_days = days
    if week == 0 and cur_days == NO_DAYS: # didn't find word in the DF dates

      days, week, todayDayWeek, dayInText, weekend, range_days = get_delta_days_weekday(text.lower(), dayInText, weekend, range_days, disp) # check day in week and return num of days and week

      if disp: print("2." + f_string.format(days, week, found_days, cur_days))
      if days != NO_DAYS and days != found_days and found_days != NO_DAYS:
        days = NO_DAYS
      elif found_days == NO_DAYS:
        found_days = days
      if week == 0 or days == NO_DAYS:
        sav_days = days

        days, week = get_number_period(doc, disp) # check number of days/weeks
        if days == NO_DAYS and sav_days != NO_DAYS:
          days = sav_days
        if disp: print("3." + f_string.format(days, week, found_days, cur_days))
        # in english the week ends on Sunday, so if we r on:
        # sunday then next_week=0 weeks from now
        # monday then next_week=1 weeks from now
        # -> we calc days up to sunday, and add 1 week

  if disp: print("4." + f_string.format(days, week, found_days, cur_days))
  if disp: p('todayDayWeek', todayDayWeek, dayInText, days, weekend)
  org_days = days
  if todayDayWeek == 0 or (days == found_days == cur_days == -dayInText == NO_DAYS):  # no day information -> today
    if TODAY == 0:
      dt = datetime.now(tz)
      days = dt.weekday() + 2 if dt.weekday() > 1 else 7 # from day of the week such that sunday=1,.. saturday=7
    else:
      days = TODAY
    todayDayWeek = dayInText = days
    found_days = days = 0 # to convert to #days from saturday (0 if saturday, 1=sunday etc)
    if disp: p(todayDayWeek, dayInText, days, found_days)
  if days != NO_DAYS and days != found_days and found_days != NO_DAYS:
    days = NO_DAYS
  elif days != NO_DAYS and found_days == NO_DAYS:
    found_days = days
  elif days == NO_DAYS and week > 0:
    days = found_days
    if days == NO_DAYS:
      days = 0 # no day is specified so days = 0 (number of days since today)
  if days != NO_DAYS:
    if disp: p(todayDayWeek, dayInText)
    # convert to english weekday number (monday=1... sunday=7)
    todayDayWeek = todayDayWeek - 1 if todayDayWeek > 1 else 7 # from our day sunday=1,.. saturday=7
    dayInText =  dayInText - 1 if dayInText > 1 else 7
    if disp: p(todayDayWeek, dayInText, week, days)    
    if todayDayWeek > dayInText and week > 0:
      week -= 1
    if range_days == 2 and weekend:
      if disp: p('weekend', org_days, org_days)
      if org_days == NO_DAYS or dayInText >= 6:
        if dayInText == 7: # if sunday there is one day left in the weekend
          if week == 1: # next week
            days -= 1 # nextweekent starts on Saturday
          else:
            range_days = 1
        elif dayInText < 6: # not sunday and not staurday
          days = 7 - dayInText # number of days to saturday
      else:
        days = NO_DAYS  # can not say 'monday, next weekend
    return week * 7 + days, range_days
  return NO_DAYS, 0
#check_date(["sunday","next month"], True)
#p('A:', check_date(["this weekend"], False))
#p('A:', check_date(["weekend"], False))
#p('A:', check_date(["next weekend"], False))
#p('A:', check_date(["current"], False))
#p('A:', check_date(["currently"], True))
#p('A:', check_date(["test"], True))
check_date(get_parts("weather tomorrow in tel aviv", False)['date'], True)


text = tomorrow
today []
tomorrow [(3573583789758258062, 0, 1)]
col: tomorrow
matchFound tomorrow, 


(2, 1)

In [210]:
TODAY = 0 # so can test different days

In [134]:
if TODAY == 0:
  dt = datetime.now(tz)
  dayofweek = list(dayInWeek.keys())[list(dayInWeek.values()).index((dt.weekday() + 1) % 7 + 1)]
  ndays = (dt.weekday() + 1) % 7 + 1
else:
  dayofweek = list(dayInWeek.keys())[list(dayInWeek.values()).index(TODAY)]
  ndays = (TODAY + 1) % 7 + 1

p(f'today is {dayofweek}')#, checking for {day}, {ndays}')

for day in dayInWeek.keys():
  print(day, '   \tA:', check_date([day,"next week"], False), end=', ')
  print('B:', check_date([day,"in one week"], False), end=', ')
  print('C:', check_date([day,"2 days"], False), end=', ')
  print('D:', check_date([day,"one week"], False), end=', ')
  print('E:', check_date([day,"1 week"], False), end=', ')
  print('F:', check_date([day,"one day"], False), end=', ')
  print('G:', check_date([day], False), end=', ')
  print('H:', check_date([day,"following week"], False), end=', ')
  print('I:', check_date([day,"week later"], False), end=', ')
  print('J:', check_date([day,"coming week"], False), end=', ')
  print('K:', check_date([day,"this week"], False), end=', ')
  print('L:', check_date([day,"this weekend"], False), end=', ')
  print('M:', check_date([day,"next weekend"], False), end=', ')
  print('N:', check_date([day,"weekend"], False), end=', ')
  
  if day == dayofweek:
    p()
    print('\ta:', check_date(["next week"], False), end=', ')
    print('b:', check_date(["in one week"], False), end=', ')
    print('c:', check_date(["2 days"], False), end=', ')
    print('d:', check_date(["one week"], False), end=', ')
    print('e:', check_date(["1 week"], False), end=', ')
    print('f:', check_date(["one day"], False), end=', ')
    print('g:', check_date(["following week"], False), end=', ')
    print('h:', check_date(["week later"], False), end=', ')
    print('i:', check_date(["coming week"], False), end=', ')
    print('j:', check_date(["this week"], False), end=', ')
    print('k:', check_date([day,"this weekend"], False), end=', ')
    print('l:', check_date([day,"next weekend"], False), end=', ')
    print('m:', check_date([day,"weekend"], False), end=', ')
  p()
p()

today is monday
sunday    	A: (13, 1), B: (13, 1), C: (-999, 0), D: (13, 1), E: (13, 1), F: (-999, 0), G: (6, 1), H: (13, 1), I: (13, 1), J: (13, 1), K: (6, 1), L: (6, 1), M: (12, 2), N: (-999, 0), 
monday    	A: (7, 1), B: (7, 1), C: (-999, 0), D: (7, 1), E: (7, 1), F: (-999, 0), G: (0, 1), H: (7, 1), I: (7, 1), J: (7, 1), K: (0, 1), L: (-999, 2), M: (13, 2), N: (-999, 0), 
	a: (7, 1), b: (7, 1), c: (2, 1), d: (7, 1), e: (7, 1), f: (1, 1), g: (7, 1), h: (7, 1), i: (7, 1), j: (0, 1), k: (-999, 2), l: (13, 2), m: (-999, 0), 
tuesday    	A: (8, 1), B: (8, 1), C: (-999, 0), D: (8, 1), E: (8, 1), F: (1, 1), G: (1, 1), H: (8, 1), I: (8, 1), J: (8, 1), K: (1, 1), L: (-999, 2), M: (12, 2), N: (-999, 0), 
wednesday    	A: (9, 1), B: (9, 1), C: (2, 1), D: (9, 1), E: (9, 1), F: (-999, 0), G: (2, 1), H: (9, 1), I: (9, 1), J: (9, 1), K: (2, 1), L: (-999, 2), M: (11, 2), N: (-999, 0), 
thursday    	A: (10, 1), B: (10, 1), C: (-999, 0), D: (10, 1), E: (10, 1), F: (-999, 0), G: (3, 1), H: (10, 1), I:

In [38]:
if False:
  '''
today is saturday
sunday    	A: 8, B: 8, C: -999, D: 8, E: 8, F: 1, G: 1, H: 8, I: 8, J: 8, K: 1, 
monday    	A: 2, B: 2, C: 2, D: 2, E: 2, F: -999, G: 2, H: 2, I: 2, J: 2, K: -999, 
tuesday    	A: 3, B: 3, C: -999, D: 3, E: 3, F: -999, G: 3, H: 3, I: 3, J: 3, K: -999, 
wednesday    	A: 4, B: 4, C: -999, D: 4, E: 4, F: -999, G: 4, H: 4, I: 4, J: 4, K: -999, 
thursday    	A: 5, B: 5, C: -999, D: 5, E: 5, F: -999, G: 5, H: 5, I: 5, J: 5, K: -999, 
friday    	A: 6, B: 6, C: -999, D: 6, E: 6, F: -999, G: 6, H: 6, I: 6, J: 6, K: -999, 
saturday    	A: 7, B: 7, C: -999, D: 7, E: 7, F: -999, G: 0, H: 7, I: 7, J: 7, K: 0, 
	a: 7, b: 7, c: 2, d: 7, e: 7, f: 1, g: 7, h: 7, i: 7, j: 0, 

today is sunday
sunday    	A: 7, B: 7, C: -999, D: 7, E: 7, F: -999, G: 0, H: 7, I: 7, J: 7, K: 0, 
	a: 7, b: 7, c: 2, d: 7, e: 7, f: 1, g: 7, h: 7, i: 7, j: 0, 
monday    	A: 1, B: 1, C: -999, D: 1, E: 1, F: 1, G: 1, H: 1, I: 1, J: 1, K: -999, 
tuesday    	A: 2, B: 2, C: 2, D: 2, E: 2, F: -999, G: 2, H: 2, I: 2, J: 2, K: -999, 
wednesday    	A: 3, B: 3, C: -999, D: 3, E: 3, F: -999, G: 3, H: 3, I: 3, J: 3, K: -999, 
thursday    	A: 4, B: 4, C: -999, D: 4, E: 4, F: -999, G: 4, H: 4, I: 4, J: 4, K: -999, 
friday    	A: 5, B: 5, C: -999, D: 5, E: 5, F: -999, G: 5, H: 5, I: 5, J: 5, K: -999, 
saturday    	A: 6, B: 6, C: -999, D: 6, E: 6, F: -999, G: 6, H: 6, I: 6, J: 6, K: -999, 

today is monday
sunday    	A: 13, B: 13, C: -999, D: 13, E: 13, F: -999, G: 6, H: 13, I: 13, J: 13, K: 6, 
monday    	A: 7, B: 7, C: -999, D: 7, E: 7, F: -999, G: 0, H: 7, I: 7, J: 7, K: 0, 
	a: 7, b: 7, c: 2, d: 7, e: 7, f: 1, g: 7, h: 7, i: 7, j: 0, 
tuesday    	A: 8, B: 8, C: -999, D: 8, E: 8, F: 1, G: 1, H: 8, I: 8, J: 8, K: 1, 
wednesday    	A: 9, B: 9, C: 2, D: 9, E: 9, F: -999, G: 2, H: 9, I: 9, J: 9, K: 2, 
thursday    	A: 10, B: 10, C: -999, D: 10, E: 10, F: -999, G: 3, H: 10, I: 10, J: 10, K: 3, 
friday    	A: 11, B: 11, C: -999, D: 11, E: 11, F: -999, G: 4, H: 11, I: 11, J: 11, K: 4, 
saturday    	A: 12, B: 12, C: -999, D: 12, E: 12, F: -999, G: 5, H: 12, I: 12, J: 12, K: 5, 
  '''   

## check city for (POS)

In [39]:
def check_gpe(entity, ents, ent_text, gpe, multiple_cities):
        token = None
        for e in ents:
          if ent_text == e[0]:
            token = e[1] 
            break
        # verify the GPE is a true GPE for cases that a city has a name that has another meaninig
        if token and token.text != token.lemma_: # a true GPE cannot be lemmatized
          return 'continue', gpe, 0

        # for example there is a city in Denemark that has a name 'rain'
        if token and len(ents) > 1 and (token.head == token or token.pos_ not in ['PROPN', 'PRON', 'NOUN']) and not entity._.is_country:
          valid_gpe = False
          for e in ents:
            if ent_text != e[0]:
              tk = e[1]
              if tk.head == token:  # the token is connected to another gpe, so it is a valid gpe (for example, (paris, canada) are connected
                valid_gpe = True
                return 'break', gpe, 0
          if not valid_gpe:
            if DEBUG: ic('A:', ent_text)
            return 'continue', gpe, 0

        ent_type = 'state'
        if DEBUG: ic('0:', ent_text, entity._.is_city, entity._.is_country)
        if entity._.is_city and not entity._.is_country: # 'canada' is a country and also a city in Portugal
          ent_type = 'city'
          if ent_text in df_CITIES_API.index: 
            code_country = df_CITIES_API.loc[ent_text].country
            if type(code_country) != str:
              multiple_cities = True
              # see note why not to use code_country[0] which sometimes give 'Key Error'
              # https://stackoverflow.com/questions/46153647/keyerror-0-when-accessing-value-in-pandas-series            
              ent_country = countries_df.loc[code_country.iloc[0]].Name
            else:
              ent_country = countries_df.loc[code_country].Name
          #elif ent_text in CITIES_IL:
          #  ent_country = 'israel'
          else:
            ent_country = ''
        elif entity._.is_country or ent_text in CITIES_API_country_code:
          ent_type = 'country'

        if ent_type not in gpe:
          gpe[ent_type] = []
        if ent_type == 'city':
           gpe[ent_type].append((entity.text, ent_country))
        else:
          gpe[ent_type].append(entity.text)
        return None, gpe, multiple_cities

In [40]:
def get_city_country(multiple_cities, gpe):
  if DEBUG: ic('1:', multiple_cities, gpe)
  if multiple_cities or ('city' in gpe and len(gpe['city']) > 1):
    if 'country' in gpe:
      # verify country code of the city is correct, when there are multiple cities
      for i, (city, city_country) in enumerate(gpe['city']):
        country = gpe['country'][0]
        if city_country != country:
          for code_country in df_CITIES_API.loc[city].country:
            if city_country == countries_df.loc[code_country].Name:
              # found a matching country
              gpe['city'][i] = (city, country)
              break
    else:
      check_capital = True
      if len(gpe['city']) == 2: # maybe entered a country which has city with that name (eg. Toronto, US = there is a city name 'us' in france and 'usa' in japan)
        # 'city': [('toronto', 'Canada'), ('us', 'france')]
        for i in range(1,-1,-1):
          city = gpe['city'][i][0].upper()
          if city in countries_df.index: #usually the 2nd "city" is the country
            city_country = countries_df.loc[city].Name
            j = 0 if i == 1 else 1
            gpe['city'] = [(gpe['city'][j][0], city_country)]
            gpe['country'] = city_country
            check_capital = False
            break
      if check_capital:
        # check if one of the cities is a capital or a large city, then use it
        #gpe, multiple_cities)
        for i, (city, city_country) in enumerate(gpe['city']):
          if city in capitals_df.index.values:
            # found a matching country
            city_country = capitals_df.loc[city].country
            gpe['city'][i] = (city, city_country)
            break
          if city in largest_df.index.values:
            # found a matching country
            city_country = largest_df.loc[city].country
            gpe['city'][i] = (city, city_country)
            break
  elif 'country' in gpe and 'city' in gpe:  # country with city, verify correct
      if DEBUG: p('2:', gpe)
      # verify country code of the city is correct, when user entered 
      city, city_country = gpe['city'][0]
      country = gpe['country'][0]
      if city_country != country:
        gpe['city'] = [(city, "ERROR: " + MESSAGES[LANG]["messages"]["city_error"].format(city, country))]
  elif 'country' in gpe and 'city' not in gpe:  # country without city, get the capital
    country = gpe['country'][0]
    #country)
    if country in CAPITALS:
      gpe['city'] = [(CAPITALS[country], country)]
  return gpe

## get parts

In [184]:
DEBUG = False
def get_parts(text, disp=False, verbose=True):
  global DEBUG
  doc = nlp(text.lower())
  #doc = nlp(text.lower().translate(space_punct_dict))
  date, time, gpe, ents = [], [], {'action':[]}, []
  multiple_cities = False
  if disp: p()
  for token in doc:
    if disp:
      p(f"Tokens of [{token.text}]: dep_={token.dep_}, ent_type={token.ent_type_}, head={token.head}, lemma_={token.lemma_}, pos_={token.pos_}, tag_={token.tag_}, is_action={token._.is_action}")
    if token._.is_action:  # checking for action first catch cities with the same name ('rain' is a city)
      tag = ACTIONS[token.lemma_]
      tag_exists = [t for t in gpe['action'] if t[0] == tag]
      if len(tag_exists) == 0:
        gpe['action'].append((tag, token.lemma_))
    elif token.ent_type_ == 'GPE': ents.append((token.text.lower(), token))
  for entity in doc.ents:
      if disp:
        p(f"Entities of [{entity.text}]: {entity.label_}='{spacy.explain(entity.label_)}', is_city={entity._.is_city}, is_country={entity._.is_country}")
      ent_text = entity.text.lower().translate(remove_punct_dict)
      if entity.label_ == 'DATE': date.append(ent_text.lower())
      if entity.label_ == 'TIME': time.append(ent_text.lower())
      '''
      if entity._.is_action:  # 'rain' is both a city and weather action
        tag = ACTIONS[ent_text]
        gpe['action'].append((tag, ent_text))
      '''
      if entity.label_ == 'GPE':
        act, gpe, multiple_cities = check_gpe(entity, ents, ent_text, gpe, multiple_cities)
        if act == 'continue': continue
        if act == 'break': break
     
  gpe = get_city_country(multiple_cities, gpe)
    # remove city with same name as action ('rain') if there is another city
  if 'city' in gpe:
    org_gpe = gpe['city'].copy() # copy because the list is modified inside the loop
    for tup in org_gpe:
      city = tup[0]
      city_exists = [t for t in gpe['action'] if t[0] == city or t[1] == city]
      if len(city_exists) == 0: continue
      if len(gpe['city']) == 1:  # must leave at least one city
        # remove from action list if appears only once in the sentence
        n = [t for t in doc if t.text == city]
        if len(n) > 1: continue
        gpe['action'].remove(city_exists[0])
        continue
      gpe['city'].remove(tup)

#  if len(date) == 0:
#    date = 'current'
  if len(gpe['action']) > 0:
    #gpe['action'] = [('all', 'weather')]
    #else:
    for i, tup in enumerate(gpe['action']):
      if tup[0] == 'all':
        if i == 0: break
        gpe['action'].remove(tup)
        gpe['action'].insert(0, tup)
        break

  result = {'date':date, 'time':time}
  result.update(gpe)

  if disp:
    p(result)
  elif verbose and 'city' not in result:
    p(f'FAILED: org [{text}], {result}')
  return result

#DEBUG = False
if TEST:
  get_parts("Today weather in a Ra-anana")
  get_parts("Today weather in a Ra'anana")  
  get_parts("Today weather in a Raanana")
  get_parts("weather in toronto") # there is a city toronto in USA and toronto is not the capital of Canada so we take the first city in the db
  get_parts("weather in toronto, canada")
  get_parts("weather paris next 5 days")
  #get_parts("tell me how is the weather in the ra- anana in the")
  get_parts("rain in toronto")
  get_parts("weather in toronto, us")
  get_parts("weather in toronto, canada")
  get_parts("weather in jenin")
  get_parts("weather in jenin, israel")
  get_parts('haifa, israel')
  DEBUG = False
  get_parts("Wheather forecast in Tel aviv")
  get_parts("paris Canada")
  get_parts("heat in paris")
  get_parts("clouds, heat, rain in toronto")
  get_parts("clouds, heat, rain, temp, pressure, humidity, wind, visibility, sunrise in toronto")
  get_parts("clouds, heat and rain in rain")
#answer = get_weather('current', [('jerusalem', 'israel')], [('clouds', 'cloud'), ('temp', 'temp'), ('pressure', 'pressure'), ('humidity', 'humidity'), ('wind', 'wind'), ('visibility', 'wind'), ('sunrise', 'wind'), ('weather','rain')])
#answer = get_weather('current', [('jerusalem', 'israel')], [('clouds', 'cloud'), ('temp', 'temp'), ('pressure', 'pressure'), ('humidity', 'humidity'), ('wind', 'wind'), ('visibility', 'wind'), ('sunrise', 'wind'), ('weather','rain')])
#answer
  get_parts("rain, clouds, weather in toronto, canada", True)
  get_parts("rain, clouds, weather in toronto", True)

  l = get_parts('in one week')['date']
  p(l, check_date(l))
  l = get_parts("friday in one week", True)['date']
  p(l, check_date(l))
  l = get_parts("friday in 1 week", True)['date']
  p(l, check_date(l))
  l = get_parts("in 1 week", True)['date']
  p(l, check_date(l))
  # not recognized by spacy: now, current, right now 
get_parts("weather week in tel aviv", True)  


Tokens of [weather]: dep_=compound, ent_type=DATE, head=week, lemma_=weather, pos_=NOUN, tag_=NN, is_action=True
Tokens of [week]: dep_=ROOT, ent_type=DATE, head=week, lemma_=week, pos_=NOUN, tag_=NN, is_action=False
Tokens of [in]: dep_=prep, ent_type=, head=week, lemma_=in, pos_=ADP, tag_=IN, is_action=False
Tokens of [tel]: dep_=compound, ent_type=GPE, head=aviv, lemma_=tel, pos_=PROPN, tag_=NNP, is_action=False
Tokens of [aviv]: dep_=pobj, ent_type=GPE, head=in, lemma_=aviv, pos_=PROPN, tag_=NNP, is_action=False


<IPython.core.display.Javascript object>

Entities of [weather week]: DATE='Absolute or relative dates or periods', is_city=False, is_country=False


<IPython.core.display.Javascript object>

Entities of [tel aviv]: GPE='Countries, cities, states', is_city=True, is_country=False
{'date': ['weather week'], 'time': [], 'action': [('all', 'weather')], 'city': [('tel aviv', 'israel')]}


{'action': [('all', 'weather')],
 'city': [('tel aviv', 'israel')],
 'date': ['weather week'],
 'time': []}

In [42]:
get_parts("20/10/2021", True)
get_parts("20/10/2021", True)
get_parts("20 10 2021", True)


Tokens of [20/10/2021]: dep_=ROOT, ent_type=, head=20/10/2021, lemma_=20/10/2021, pos_=NUM, tag_=CD, is_action=False
{'date': 'current', 'time': [], 'action': []}

Tokens of [20/10/2021]: dep_=ROOT, ent_type=, head=20/10/2021, lemma_=20/10/2021, pos_=NUM, tag_=CD, is_action=False
{'date': 'current', 'time': [], 'action': []}

Tokens of [20]: dep_=ROOT, ent_type=DATE, head=20, lemma_=20, pos_=NUM, tag_=CD, is_action=False
Tokens of [10]: dep_=nummod, ent_type=DATE, head=2021, lemma_=10, pos_=NUM, tag_=CD, is_action=False
Tokens of [2021]: dep_=quantmod, ent_type=DATE, head=20, lemma_=2021, pos_=NUM, tag_=CD, is_action=False


<IPython.core.display.Javascript object>

Entities of [20 10 2021]: DATE='Absolute or relative dates or periods', is_city=False, is_country=False
{'date': ['20 10 2021'], 'time': [], 'action': []}


{'action': [], 'date': ['20 10 2021'], 'time': []}

In [43]:
if TEST:
  get_parts("paris Canada")
  get_parts("las vegas, united states")
  get_parts("las vegas, US")  
  get_parts("weather in las vegas, united states")
  get_parts("weather in las vegas, US")
  get_parts("weather in new york")    
  get_parts("weather in new york us")
get_parts("paris, Canada")

FAILED: org [paris, Canada], {'date': 'current', 'time': [], 'action': []}


{'action': [], 'date': 'current', 'time': []}

In [44]:
DEBUG = False
if TEST:
  get_parts("please tell me if it will rain tonight in Rain")
  get_parts("tomorrow and today rain in spain")
  get_parts("tomorrow rain in madrid")
  get_parts("tomorrow rain in beersheba")
  get_parts("tomorrow rain in israel")
  get_parts("weather paris Canada")
  get_parts("weather paris, Canada")
  get_parts("tomorrow rain in yafo")
  

In [45]:
if TEST:
  get_parts("weather in new york")
  get_parts("weather in new york us")
  get_parts("weather in las vegas")
  get_parts("weather in las vegas, US")
  get_parts("weather in las vegas, united states")
  get_parts("weather in kas vegas")
  get_parts("weather in kas vegas, US")
  get_parts("what is the weather in summer, Israel")
# ask before switching a langauge
get_parts("weather in las vegas, us", True)
get_parts("weather in new england, us", True)


Tokens of [weather]: dep_=ROOT, ent_type=, head=weather, lemma_=weather, pos_=NOUN, tag_=NN, is_action=True
Tokens of [in]: dep_=prep, ent_type=, head=weather, lemma_=in, pos_=ADP, tag_=IN, is_action=False
Tokens of [las]: dep_=compound, ent_type=GPE, head=vegas, lemma_=las, pos_=PROPN, tag_=NNP, is_action=False
Tokens of [vegas]: dep_=pobj, ent_type=GPE, head=in, lemma_=vegas, pos_=PROPN, tag_=NNP, is_action=False
Tokens of [,]: dep_=punct, ent_type=, head=weather, lemma_=,, pos_=PUNCT, tag_=,, is_action=False
Tokens of [us]: dep_=npadvmod, ent_type=GPE, head=weather, lemma_=us, pos_=PROPN, tag_=NNP, is_action=False


<IPython.core.display.Javascript object>

Entities of [las vegas]: GPE='Countries, cities, states', is_city=True, is_country=False


<IPython.core.display.Javascript object>

Entities of [us]: GPE='Countries, cities, states', is_city=True, is_country=False
{'date': 'current', 'time': [], 'action': [('all', 'weather')], 'city': [('las vegas', 'united states')], 'country': 'united states'}

Tokens of [weather]: dep_=ROOT, ent_type=, head=weather, lemma_=weather, pos_=NOUN, tag_=NN, is_action=True
Tokens of [in]: dep_=prep, ent_type=, head=weather, lemma_=in, pos_=ADP, tag_=IN, is_action=False
Tokens of [new]: dep_=compound, ent_type=, head=england, lemma_=new, pos_=PROPN, tag_=NNP, is_action=False
Tokens of [england]: dep_=pobj, ent_type=GPE, head=in, lemma_=england, pos_=PROPN, tag_=NNP, is_action=False
Tokens of [,]: dep_=punct, ent_type=, head=england, lemma_=,, pos_=PUNCT, tag_=,, is_action=False
Tokens of [us]: dep_=pobj, ent_type=GPE, head=in, lemma_=us, pos_=PROPN, tag_=NNP, is_action=False


<IPython.core.display.Javascript object>

Entities of [england]: GPE='Countries, cities, states', is_city=True, is_country=False


<IPython.core.display.Javascript object>

Entities of [us]: GPE='Countries, cities, states', is_city=True, is_country=False
{'date': 'current', 'time': [], 'action': [('all', 'weather')], 'city': [('england', 'united states')], 'country': 'united states'}


{'action': [('all', 'weather')],
 'city': [('england', 'united states')],
 'country': 'united states',
 'date': 'current',
 'time': []}

In [46]:
if TEST:
  get_parts("Today weather in a Ra-anana")
  get_parts("weather in tel aviv")
  get_parts("tomorrow weather in haifa")
  get_parts("tomorrow rain in beersheba")
  get_parts("weather paris today and tomorrow")
  get_parts("weather paris")
  get_parts("weather paris in December")
  get_parts("ra'anana")
  get_parts("Today weather in a Tel Aviv")
  get_parts("weather paris")
  get_parts("paris weather ")
  get_parts('Paris weather in')
  get_parts("weather shelomi")

In [47]:
if TEST:
  # example of successful chat although get parts failed
  get_parts("weather in tel-aviv")
  get_parts("weather in TelAviv")   # gave answer for Israel/Jerusalem
  get_parts("what is tel aviv")
  get_parts("wether in tel aviv")
  get_parts("please tell me if it will rain tonight in Rain")

In [48]:
if TEST:
  # fixed in round1 of the bot (remove stop words and dashes followed by a space '- ')
  get_parts("weather ra- anana") # for google
  get_parts("tell me how is the weather in the ra- anana in the")

# Run the bot

> when similarity was .075 it matched the sentence

>"tell me how is the weather in the ra- anana in the"

>to "what's up" of tag ["greeting"] so we increased it to 0.85

## Google search

In [49]:
def get_google(text):
  url = "https://google.com/search?q=" + text + '&hl=en'
  request_result = requests.get(url)
  soup = bs4.BeautifulSoup(request_result.text, "html.parser")
  spans = soup.find_all('span', class_='BNeawe')
  new_text = ''
  for span in spans: 
    if isinstance(span.contents[0], bs4.NavigableString):
      new_text += str(span.contents[0]) + ' '
  return new_text
get_google("'קשאיקר ןמ ןדרשקך")

'Tel Aviv-Yafo, Israel  /  Weather '

In [50]:
def add_language(lang):
  lang_exists = [t for t in MESSAGES_lang if t == lang]
  if len(lang_exists) > 0:
    p(f'{lang} already exists')
    return
  MESSAGES_lang.append(lang)
  p(MESSAGES_lang)
  MESSAGES.update({lang:{'messages':{}}})
  new_msg = list(translator.translate(list(MESSAGES_en.values()), dest=lang))
  for i, k in enumerate(MESSAGES_en.keys()):
    if type(new_msg[i]) is list:
      for msg in new_msg[i]:
        MESSAGES[lang]["messages"].update({k: msg.text})
      continue
    MESSAGES[lang]["messages"].update({k: new_msg[i].text})
#  MESSAGES[lang])
  new_msg = translator.translate(list(intents_en), dest=lang)
  intents.update({lang:{'messages':[]}})
  msg_arr, i = [], 0
  for dict_ in intents['en']['messages']:
    resp, msg = [], {}
    for k, v in dict_.items():
      if k == 'tag' or k == 'patterns':
        msg.update({k: v})
        continue
      for j in range(len(intents['en']["messages"][i]["responses"])):
        resp.append(new_msg[i][j].text)
    msg.update({"responses": resp})
    msg_arr.append(msg)    
    i += 1
  intents[lang].update({"messages": msg_arr})
  # save the new langauge
  new_d = {"intents": {"language": {lang: {'messages': {}}}},
          "messages": {"language": {lang: {'messages': {}}}}}
  new_d['intents']['language'][lang] = intents[lang]
  new_d['messages']['language'][lang] = MESSAGES[lang]
  # read old new langaused that were not yet added to main messages file
  list_d = []
  try:
    with open(path + 'new_messages.json', "r") as f:
      list_d = json.loads(f.read())
  except Exception as error:
    pass
    #error)
  list_d.insert(0, new_d) #append(new_d)
  #list_d)
  with open(path + 'new_messages.json', "w") as f:
    json.dump(list_d, f, indent=1, ensure_ascii = False)
    
#translator.detect('الشغل'))

# 'xab' is in Hmong langauge (hmn) 
#translator.detect('Работа'))
#add_language('iw') # hebrew
#add_language('ar') # arabic
#add_language('ru') # russian
#add_language('fr')

## Send email

SMTPAuthenticationError: (534, b'5.7.14 <https://accounts.google.com/signin/continue?sarp=1&scc=1&plt=AKgnsbu\n5.7.14 wikqxaraOBaoJ8uaTDqCVoXmzPp36d_A6D8Ay2dQ-cRIGSVS1O88IW5d5hVoC21k64yZ8\n5.7.14 zt8t4BLQx1zO_S0arOXBdjPX4pd3KRLEhWoYyXJorkcAkitmYm1SAhepjia4Eep4>\n5.7.14 Please log in via your web browser and then try again.\n5.7.14  Learn more at\n5.7.14  https://support.google.com/mail/answer/78754 m13sm2830102vsl.16 - gsmtp')

In [51]:
import smtplib
from email.message import EmailMessage
msg = EmailMessage()

# locally, if fails to send open google 'unlock captcha' and activate it https://accounts.google.com/b/0/DisplayUnlockCaptcha

def send_email(text='Mmessage from WeatherBot local', subject='New Session', to="drbarak@talkie.co.il"):
  msg = EmailMessage()

  msg.set_content('')
  # Add the html version.  This converts the message into a multipart/alternative
  # container, with the original text message as the first part and the new html
  # message as the second part. (Note that the first part of the messge did not work for me and didnot displaed on the incomin email)
  msg.add_alternative('''
  <html>
    <head>
      <h1>{subject}</h1>
    </head>
    <body>
      {text}
    </body>
  </html>
  '''.format(subject=subject, text=text), subtype='html')
  msg['Subject'] = subject
  msg['From'] = "Weatherbot <dr.zvibarak@gmail.com>"
  msg['To'] = to

  try:
    smtpObj = smtplib.SMTP('smtp.gmail.com', 587)
    smtpObj.starttls()
    smtpObj.login('dr.zvibarak@gmail.com', 'shushu1952')
    smtpObj.send_message(msg)
    smtpObj.quit()
    print("Successfully sent email")
    return True
  except Exception as e:# SMTPException:
    print("Error: unable to send email")
    p(e)
    return False
#send_email(text="user_msg", subject='Log from WeatherBot', to="drbarak@talkie.co.il")

In [233]:
GOOGLE = True
do = True
GREETING, NOANSWER = 0, 3
#intents['intents'][GREETING]['responses'][0])
min_similarity = 0.84
VERBOSE = 0
LANG = 'en'
new_lang = ''
RUN_TEST = 0

while do:

  if RUN_TEST > 0:
    user_msg = QUESTIONS[RUN_TEST- 1]
    p(user_msg)
    RUN_TEST += 1
    VERBOSE = 0
    if RUN_TEST > len(QUESTIONS): RUN_TEST = 0
  else:
    user_msg = input().lower().strip()
  if user_msg == '':
    answers = intents[LANG]["messages"][NOANSWER]['responses']
    p("WeatherBot: " + random.choice(answers))
    continue
  if user_msg == 'run test':
    RUN_TEST = 1
    continue

  switched_lang = False
  if new_lang != '':
    if user_msg == '1':
      if new_lang not in MESSAGES_lang or new_lang=='fr':
        add_language(new_lang)
      LANG = new_lang
      switched_lang = True
    user_msg = org_msg
    new_lang = ''
  elif not RUN_TEST:
    response = translator.detect(user_msg)
    if response.lang == 'iw': response.lang = 'he'
    if response.lang != LANG:
      p(response.lang)
      if response.lang in ['en','he']:
        LANG = response.lang
        switched_lang = True
      else:
        verify = True
        if response.lang not in MESSAGES_lang:
            # add langauge only when sure - of at least 3 words of length 5 letters each - to prevent erroneous detection
          sentence = ' '.join(filter(None, user_msg.split(' ')))
          if not (len(sentence) > 11 and len(user_msg.split()) > 2):
            verify = False  # False "new langauge" - google think that 'haifa israel' is in arabic, and 'Paris, frnace' is in french
        if verify:
          new_lang = response.lang
          org_msg = user_msg
          p("WeatherBot: " + MESSAGES[LANG]["messages"]["new_lang"].format(new_lang))
          continue
    elif LANG == 'en': # check if the google translate stopped working
      test = translator.detect('שלום')
      if test.lang == LANG:
        #p('Note: Google translate stopped working temporarily -> multi-lingual option is not functional')
        pass

  round = 0
  answer = ''
  org_msg = user_msg
  tag = ''
  while round < 5 and do:
    if VERBOSE: p(f'round {round}: {user_msg}')
    doc1 = nlp(user_msg)
    answers, tag = None, None
    top_similarity = 0
    # verify not an erroneous finding (if we have GPE in the doc we assume it is not a general request)
    '''
    for token in doc1:
      p(f"Tokens of [{token.text}]: dep_={token.dep_}, ent_type={token.ent_type_}, head={token.head}, lemma_={token.lemma_}, pos_={token.pos_}, tag_={token.tag_}, is_action={token._.is_action}, like_email ={token.like_email }")
    for entity in doc1.ents:
      p(f"Entities of [{entity.text}]: {entity.label_}='{spacy.explain(entity.label_)}', is_city={entity._.is_city}, is_country={entity._.is_country}")
    '''
    if len([ent for ent in doc1.ents if ent.label_ == 'GPE']) == 0 or len([tk for tk in doc1 if tk.like_email]) > 0:
      for ints in intents[LANG]["messages"]:
        #ints)
        for text in ints['patterns']:
          doc2 = nlp(text)
          # Get the similarity of doc1 and doc2
          if doc1.vector_norm == 0 or doc2.vector_norm == 0:
            continue
          similarity = doc1.similarity(doc2)
          if VERBOSE > 2: p(f"{similarity}, {doc2}, {[ints['tag']]}")
          if similarity > top_similarity:
            top_similarity = similarity
            answers = ints['responses']
            tag = ints['tag']
      #p(top_similarity, tag)
      #p(answers, tag)
      shift = 3 if tag == "mail" else 0
      if top_similarity > min_similarity - shift:
        if tag == "goodbye":
          do = False
        if tag in ["goodbye", "greeting", "thanks", "options"]:
          answer = random.choice(answers)
          break
        if tag == "mail":
          to = [tk for tk in doc1 if tk.like_email][0]
          if not send_email(text=user_msg, subject='Log from WeatherBot (local)', to=to):
            answer = f"Failed to send email to [{to}]"
          else:
            answer = random.choice(answers)
          break
    if round == 2 and GOOGLE: round = 3 # to know we got the answer from get_parts()
    parts = get_parts(user_msg, VERBOSE > 1, False)
    if VERBOSE:  p('x:', parts)
    days = NO_DAYS
    if 'date' in parts:
      days, range_days = check_date(parts['date'])
      if True or VERBOSE:  p(days, range_days)
      if days > 7: days = NO_DAYS
      else:
        if days + range_days > 8:
          range_days = 7 - days
        if range_days == 0: range_days = 1
        if True or VERBOSE:  p(days, range_days)
    if days != NO_DAYS and 'city' in parts and 'ERROR' not in parts['city'][0][1] and len(parts['action']) > 0:
      p(round, parts)
      answer, url_icon = get_weather(days, parts['city'], parts['action'], range_days, LANG)
      break
    elif 'city' in parts and 'ERROR' in parts['city'][0][1] and not switched_lang:  #to try google for gibrish
      answer = parts['city'][0][1]
      break
    elif round == 0:
      round = 1
      user_msg = ''
      space = ' '
      for token in doc1:
        if token.text in stopwords: continue
        text = token.text.lower().translate(remove_punct_dict)
        if text == '': continue
        if text[-1] == '-': # take care of 'ra- anana'
          user_msg += space + text[:-1]
          space = ''
        else:
          user_msg += space + text
          space = ' '
      continue
    elif round == 1:
      round = 2
      user_msg = translator.translate(org_msg) # default translation is to english - can set source/dest langauge: src="de"/ dest="he"
      if VERBOSE:  p(f"{user_msg.origin} ({user_msg.src}) --> {user_msg.text} ({user_msg.dest})")
      user_msg = user_msg.text
      if user_msg != '':
        continue
    if round == 2 or round == 3:  # need this round because in round 2 get_google('ביי') returns empty string
      round = 4
      user_msg = get_google(org_msg)
      #ic(round, switched_lang, user_msg, org_msg)
      if user_msg != '':
        continue
    if round > 3:
      break
  if round >= 3 and not answer == '' and not 'ERROR' in answer:
    #ic(type(answer))
    #ic(answer)
    answer += f' ({MESSAGES[LANG]["messages"]["from_google"]})'
  if answer == '':
    if 'city' not in parts or len(parts['city']) == 0:
      answer = MESSAGES[LANG]["messages"]["city_unknown"]  
    elif 'action' not in parts or len(parts['action']) == 0:
      answer = MESSAGES[LANG]["messages"]["action_unknown"] 
    else: # if 'date' not in parts or len(parts['date']) == 0:
      #answer = MESSAGES[LANG]["messages"]["date_unknown"] 
      answer = "You need to tell me for when do you want the weather information you are looking for (such as 'today', 'tomorrow', 'monday'), up to 7 days from today"
  p(f"WeatherBot: {answer}")
  if RUN_TEST:
    p()
  #ic(tag)
# weather in jenin, israel  
# 'קשאיקר ןמ ןדרשקך (weather in israel in hebrew)
# clouds and pressure in toronto
# clouds, temp, pressure, humidity in toronto
# get_parts("clouds, heat, rain, temp, pressure, humidity, wind, visibility, sunrise in toronto")
# get_parts("clouds, heat and rain in rain", True)
# send drbarak@talkie.co.il
# rain in toronto tomorrow


rain in toronto tomorrow


ic| <ipython-input-223-4b091e598a7f>:15 in call_web()
    api_url: 'http://api.openweathermap.org/data/2.5/onecall?lat=43.700111&lon=-79.416298&exclude=minutely,hourly&units=metric&lang=en&appid=c2adfa29edfd95ad16efab9218619ff3'


x: {'date': ['tomorrow'], 'time': [], 'action': [('weather', 'rain')], 'city': [('toronto', 'canada')]}
1 1
1 1
0 {'date': ['tomorrow'], 'time': [], 'action': [('weather', 'rain')], 'city': [('toronto', 'canada')]}
True
WeatherBot: In Toronto, Canada the current weather is: <br>The weather tomorrow:  no rain, broken clouds,<br>The weather in 2 days:  no rain, broken clouds,<br>The weather in 3 days:  no rain, broken clouds,<br>The weather in 4 days:  no rain, broken clouds,<br>The weather in 5 days:  no rain, broken clouds,<br>The weather in 6 days:  no rain, broken clouds,<br>The weather in 7 days:  no rain, broken clouds,<br>The weather in 8 days:  no rain, broken clouds
rain in toronto now


ic| <ipython-input-223-4b091e598a7f>:15 in call_web()
    api_url: 'http://api.openweathermap.org/data/2.5/weather?q=toronto,,CA&units=metric&lang=en&appid=c2adfa29edfd95ad16efab9218619ff3'


x: {'date': ['now'], 'time': [], 'action': [('weather', 'rain')], 'city': [('toronto', 'canada')]}
0 1
0 1
0 {'date': ['now'], 'time': [], 'action': [('weather', 'rain')], 'city': [('toronto', 'canada')]}
False
WeatherBot: In Toronto, Canada the current weather is:  no rain, broken clouds
bye
WeatherBot: See you!


# TODO:

1. DONE: display country when getting result of weather so the user knows it is the right country - if there multiple cities

2. DONE: load jsons from github - (one file too large so download it from a shared goole link on my google drive)

3. DONE: Use flask

4. DONE: if fails to understand try round 2 - removing stop words (punctuation akready done by spacy), then round 3 use google to try to fix typo mistake and only if all fails display a message to user

5. To identify if it is a question or not - must be question

# QUESTION:

2. Are we to handle 'Ra-anana'? we handle 'Raanana'
3. DONE - There is a city toronto in USA and toronto is not the capital of Canada so we take the first city in the db (we check now if it is one of the 100 largest cities in the world)

# EXAMPLES

0 You:	weather in jenin, israel

1	Bot:	ERROR: city [jenin] not found in country [israel] - try a different spelling of the city

2	You:	weather in hebron, israel

3	Bot:	In Hebron, Israel the current weather is: clear sky

4	You:	weather in jenin

5	Bot:	In Jenin, Poland the current weather is: overcast clouds

6	You:	weather in hebron

7	Bot:	In Hebron, Palestine, state of the current weather is: clear sky

8 weather isral

9 WeatherBot: In Jerusalem, Israel the current weather is: clear sky (from Google)

10 中國的天氣

In Beijing, China the current weather is: overcast clouds (from Google)

11 погода в китае
12 погода в россии

# TEST